In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import time
import requests, sys

#### STEP1: Target list

In [3]:
genes = ['ENSMUST00000000312',
         'ENSMUST00000018186',
         'ENSMUST00000027675',
         'ENSMUST00000030944',
         'ENSMUST00000037811',
         'ENSMUST00000073490',
         'ENSMUST00000099946',
         'ENSMUST00000113270',
         'ENSMUST00000171239'
        ]

#### STEP2: Retreive cDNA and CDS code from Ensembl

In [4]:
# Ensemble REST API
api_prefix = "https://rest.ensembl.org"

In [6]:
genedata = dict()
for gene in genes:
    genedata[gene] = dict()
    genesubdata = {
        'cdna': f'/sequence/id/{gene}?type=cdna',
        'cds': f'/sequence/id/{gene}?type=cds'
    }
    for k, v in genesubdata.items():
        r = requests.get(f'{api_prefix}{v}', headers={ "Content-Type" : "text/plain"})
        if not r.ok:
            print(f'error: {gene}/{k} - {r.raise_for_status()}')
            continue
        genedata[gene][k] = r.text
        print(f'{api_prefix}{v}')

d:\workspace\rnamotif\env\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rest.ensembl.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


https://rest.ensembl.org/sequence/id/ENSMUST00000000312?type=cdna


d:\workspace\rnamotif\env\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rest.ensembl.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


https://rest.ensembl.org/sequence/id/ENSMUST00000000312?type=cds


#### STEP3: Extract 3'UTR from cDNA and CDS

In [15]:
for k, v in genedata.items():
    cdna = genedata[k]['cdna']
    cds =  genedata[k]['cds']
    utr3 = cdna[cdna.find(cds) + len(cds):]
    genedata[k]['utr3'] = utr3

In [19]:
for k,v in genedata.items():
    print(f'>{k} utr3\n{genedata[k]["utr3"]}\n\n')

>ENSMUST00000000312 utr3
GGGACTAGCAAGTCTCCCCCGTGTGGCACCATGGGAGATGCAGAATAATTATATCAGTGGTCTTTCAGCTCCTTCCCTGAGTGTGTAGAAGAGAGACTGATCTGAGAAGTGTGCAGATTGCATAGTGGTCTCACTCTCCCTACTGGACTGTCTGTGTTAGGATGGTTTTCACTGATTGTTGAAATCTTTTTTTATTTTTTATTTTTACAGTGCTGAGATATAAACTGTGCCTTTTTTTGTTTGTTTGTTTCTGTTTTTGTTCTTTTGAGCTATGATCTGCCCCAGACACAACAGCCCCAAGCCCCTCACACCTCACTAATTTTTTACATTGTGTACTTGCCCTCAATTACCATGTTTGCTGTATTCTAATAGTCACTCATGTTCCTGAATTCTGTTGCCCTGCCCAGGTGATATTCTAGGATGCAGAAATGCCTGGGCCCTTTTATGGTGAGAGACAGGTATCTTGGTGTGGGTGCAACTGCGCTGGATAGTGTGTGTGTTCCCAAGATCTTTCGTGGTATTCCCTCTCCACCTCCAGAGAACTCATTTACAGTGGCATTCCTTGTTCGGCTATGTGTCTGGGGCAGAACAAAAAAAAGGGACCACTATGCATGCTGCACACGTCTCAGATTCTTAGGTACACACCTGATTCTTAGGTGCATGCCATAGTGGGATATGTTGCTTTGATCAGAACCTGCAGGGAGGTTTTCGGGCACCACTTAAGTTTCTTGGCGTTTCTTTCAAACCAAAACTAAAGAATGGTTGTTCTCTGAGAGAGACTGGAGTGCCACCACCAAAGACAGAGGAGAGAAAAGGAGAGAAACCAAACTTGGGGACAGCAACATCAGCGAACCCGGCTAGTTGGCACACCGATGGTGAGGGTACACAGGCGGTGAGACCTATCCCACAAGATTTCTGGAAGACTAGGCTTATCTCAACCAATGTTTTCTGGCTGGAATCTTTGTCCATGTATTC

In [1]:
import os

In [2]:
os.chdir('..')

In [5]:
import pandas as pd
from rnamotif.connectors.get_ensembl import get_ensembl_seq_by_gene
from rnamotif.utility.ensembl_rna_handler import save_one_fasta_from_ensembl_api_return

In [9]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
df_s1 = pd.read_excel(r'C:\Users\by003457\Downloads\aan2399_table_S1.xlsx')

In [13]:
lst_gene = list(df_s1['target_id'])
lst_gene1 = lst_gene[:5000]
lst_gene2 = lst_gene[5000:]

In [10]:
res = get_ensembl_seq_by_gene(lst_gene,seq_type='cdna',is_ssl_verification=False)

0 - https://rest.ensembl.org/sequence/id/ENSMUST00000087033?type=cdna
1 - https://rest.ensembl.org/sequence/id/ENSMUST00000178282?type=cdna
2 - https://rest.ensembl.org/sequence/id/ENSMUST00000029805?type=cdna
3 - https://rest.ensembl.org/sequence/id/ENSMUST00000103410?type=cdna
4 - https://rest.ensembl.org/sequence/id/ENSMUST00000073490?type=cdna
5 - https://rest.ensembl.org/sequence/id/ENSMUST00000206682?type=cdna
6 - https://rest.ensembl.org/sequence/id/ENSMUST00000029658?type=cdna
7 - https://rest.ensembl.org/sequence/id/ENSMUST00000029871?type=cdna
8 - https://rest.ensembl.org/sequence/id/ENSMUST00000082235?type=cdna
9 - https://rest.ensembl.org/sequence/id/ENSMUST00000175849?type=cdna
10 - https://rest.ensembl.org/sequence/id/ENSMUST00000109410?type=cdna
11 - https://rest.ensembl.org/sequence/id/ENSMUST00000166620?type=cdna
12 - https://rest.ensembl.org/sequence/id/ENSMUST00000113270?type=cdna
13 - https://rest.ensembl.org/sequence/id/ENSMUST00000178641?type=cdna
14 - https://res

116 - https://rest.ensembl.org/sequence/id/ENSMUST00000151376?type=cdna
117 - https://rest.ensembl.org/sequence/id/ENSMUST00000143083?type=cdna
118 - https://rest.ensembl.org/sequence/id/ENSMUST00000107392?type=cdna
119 - https://rest.ensembl.org/sequence/id/ENSMUST00000015278?type=cdna
120 - https://rest.ensembl.org/sequence/id/ENSMUST00000177601?type=cdna
121 - https://rest.ensembl.org/sequence/id/ENSMUST00000109542?type=cdna
122 - https://rest.ensembl.org/sequence/id/ENSMUST00000025161?type=cdna
123 - https://rest.ensembl.org/sequence/id/ENSMUST00000101295?type=cdna
124 - https://rest.ensembl.org/sequence/id/ENSMUST00000197072?type=cdna
125 - https://rest.ensembl.org/sequence/id/ENSMUST00000143074?type=cdna
126 - https://rest.ensembl.org/sequence/id/ENSMUST00000106899?type=cdna
127 - https://rest.ensembl.org/sequence/id/ENSMUST00000079957?type=cdna
128 - https://rest.ensembl.org/sequence/id/ENSMUST00000034588?type=cdna
129 - https://rest.ensembl.org/sequence/id/ENSMUST00000033824?ty

230 - https://rest.ensembl.org/sequence/id/ENSMUST00000207236?type=cdna
231 - https://rest.ensembl.org/sequence/id/ENSMUST00000110391?type=cdna
232 - https://rest.ensembl.org/sequence/id/ENSMUST00000194953?type=cdna
233 - https://rest.ensembl.org/sequence/id/ENSMUST00000072061?type=cdna
234 - https://rest.ensembl.org/sequence/id/ENSMUST00000172321?type=cdna
235 - https://rest.ensembl.org/sequence/id/ENSMUST00000087883?type=cdna
236 - https://rest.ensembl.org/sequence/id/ENSMUST00000138367?type=cdna
237 - https://rest.ensembl.org/sequence/id/ENSMUST00000025083?type=cdna
238 - https://rest.ensembl.org/sequence/id/ENSMUST00000080654?type=cdna
239 - https://rest.ensembl.org/sequence/id/ENSMUST00000171797?type=cdna
240 - https://rest.ensembl.org/sequence/id/ENSMUST00000200823?type=cdna
241 - https://rest.ensembl.org/sequence/id/ENSMUST00000048050?type=cdna
242 - https://rest.ensembl.org/sequence/id/ENSMUST00000017851?type=cdna
243 - https://rest.ensembl.org/sequence/id/ENSMUST00000034537?ty

344 - https://rest.ensembl.org/sequence/id/ENSMUST00000027290?type=cdna
345 - https://rest.ensembl.org/sequence/id/ENSMUST00000173665?type=cdna
346 - https://rest.ensembl.org/sequence/id/ENSMUST00000135569?type=cdna
347 - https://rest.ensembl.org/sequence/id/ENSMUST00000090195?type=cdna
348 - https://rest.ensembl.org/sequence/id/ENSMUST00000015894?type=cdna
349 - https://rest.ensembl.org/sequence/id/ENSMUST00000178477?type=cdna
350 - https://rest.ensembl.org/sequence/id/ENSMUST00000151252?type=cdna
351 - https://rest.ensembl.org/sequence/id/ENSMUST00000165532?type=cdna
352 - https://rest.ensembl.org/sequence/id/ENSMUST00000032073?type=cdna
353 - https://rest.ensembl.org/sequence/id/ENSMUST00000208773?type=cdna
354 - https://rest.ensembl.org/sequence/id/ENSMUST00000025844?type=cdna
355 - https://rest.ensembl.org/sequence/id/ENSMUST00000024944?type=cdna
356 - https://rest.ensembl.org/sequence/id/ENSMUST00000189430?type=cdna
357 - https://rest.ensembl.org/sequence/id/ENSMUST00000033133?ty

458 - https://rest.ensembl.org/sequence/id/ENSMUST00000161858?type=cdna
459 - https://rest.ensembl.org/sequence/id/ENSMUST00000168015?type=cdna
460 - https://rest.ensembl.org/sequence/id/ENSMUST00000071455?type=cdna
461 - https://rest.ensembl.org/sequence/id/ENSMUST00000024004?type=cdna
462 - https://rest.ensembl.org/sequence/id/ENSMUST00000033905?type=cdna
463 - https://rest.ensembl.org/sequence/id/ENSMUST00000052725?type=cdna
464 - https://rest.ensembl.org/sequence/id/ENSMUST00000054351?type=cdna
465 - https://rest.ensembl.org/sequence/id/ENSMUST00000115021?type=cdna
466 - https://rest.ensembl.org/sequence/id/ENSMUST00000070597?type=cdna
467 - https://rest.ensembl.org/sequence/id/ENSMUST00000204807?type=cdna
468 - https://rest.ensembl.org/sequence/id/ENSMUST00000034172?type=cdna
469 - https://rest.ensembl.org/sequence/id/ENSMUST00000076463?type=cdna
470 - https://rest.ensembl.org/sequence/id/ENSMUST00000016678?type=cdna
471 - https://rest.ensembl.org/sequence/id/ENSMUST00000202000?ty

...

9409 - https://rest.ensembl.org/sequence/id/ENSMUST00000102483?type=cdna
9410 - https://rest.ensembl.org/sequence/id/ENSMUST00000149719?type=cdna
9411 - https://rest.ensembl.org/sequence/id/ENSMUST00000028668?type=cdna
9412 - https://rest.ensembl.org/sequence/id/ENSMUST00000167388?type=cdna
9413 - https://rest.ensembl.org/sequence/id/ENSMUST00000171561?type=cdna
9414 - https://rest.ensembl.org/sequence/id/ENSMUST00000056398?type=cdna
9415 - https://rest.ensembl.org/sequence/id/ENSMUST00000005600?type=cdna
9416 - https://rest.ensembl.org/sequence/id/ENSMUST00000147423?type=cdna
9417 - https://rest.ensembl.org/sequence/id/ENSMUST00000140375?type=cdna
9418 - https://rest.ensembl.org/sequence/id/ENSMUST00000166123?type=cdna
9419 - https://rest.ensembl.org/sequence/id/ENSMUST00000034763?type=cdna
9420 - https://rest.ensembl.org/sequence/id/ENSMUST00000160801?type=cdna
9421 - https://rest.ensembl.org/sequence/id/ENSMUST00000006523?type=cdna
9422 - https://rest.ensembl.org/sequence/id/ENSMUST

9522 - https://rest.ensembl.org/sequence/id/ENSMUST00000189439?type=cdna
9523 - https://rest.ensembl.org/sequence/id/ENSMUST00000081567?type=cdna
9524 - https://rest.ensembl.org/sequence/id/ENSMUST00000040021?type=cdna
9525 - https://rest.ensembl.org/sequence/id/ENSMUST00000057795?type=cdna
9526 - https://rest.ensembl.org/sequence/id/ENSMUST00000133530?type=cdna
9527 - https://rest.ensembl.org/sequence/id/ENSMUST00000171237?type=cdna
9528 - https://rest.ensembl.org/sequence/id/ENSMUST00000040656?type=cdna
9529 - https://rest.ensembl.org/sequence/id/ENSMUST00000025040?type=cdna
9530 - https://rest.ensembl.org/sequence/id/ENSMUST00000041231?type=cdna
9531 - https://rest.ensembl.org/sequence/id/ENSMUST00000150553?type=cdna
9532 - https://rest.ensembl.org/sequence/id/ENSMUST00000201803?type=cdna
9533 - https://rest.ensembl.org/sequence/id/ENSMUST00000005820?type=cdna
9534 - https://rest.ensembl.org/sequence/id/ENSMUST00000155566?type=cdna
9535 - https://rest.ensembl.org/sequence/id/ENSMUST

9635 - https://rest.ensembl.org/sequence/id/ENSMUST00000013966?type=cdna
9636 - https://rest.ensembl.org/sequence/id/ENSMUST00000104994?type=cdna
9637 - https://rest.ensembl.org/sequence/id/ENSMUST00000052145?type=cdna
9638 - https://rest.ensembl.org/sequence/id/ENSMUST00000136997?type=cdna
9639 - https://rest.ensembl.org/sequence/id/ENSMUST00000022310?type=cdna
9640 - https://rest.ensembl.org/sequence/id/ENSMUST00000076084?type=cdna
9641 - https://rest.ensembl.org/sequence/id/ENSMUST00000034607?type=cdna
9642 - https://rest.ensembl.org/sequence/id/ENSMUST00000044189?type=cdna
9643 - https://rest.ensembl.org/sequence/id/ENSMUST00000136448?type=cdna
9644 - https://rest.ensembl.org/sequence/id/ENSMUST00000200692?type=cdna
9645 - https://rest.ensembl.org/sequence/id/ENSMUST00000027623?type=cdna
9646 - https://rest.ensembl.org/sequence/id/ENSMUST00000007993?type=cdna
9647 - https://rest.ensembl.org/sequence/id/ENSMUST00000030797?type=cdna
9648 - https://rest.ensembl.org/sequence/id/ENSMUST

9749 - https://rest.ensembl.org/sequence/id/ENSMUST00000107503?type=cdna
9750 - https://rest.ensembl.org/sequence/id/ENSMUST00000178344?type=cdna
9751 - https://rest.ensembl.org/sequence/id/ENSMUST00000109455?type=cdna
9752 - https://rest.ensembl.org/sequence/id/ENSMUST00000148408?type=cdna
9753 - https://rest.ensembl.org/sequence/id/ENSMUST00000141435?type=cdna
9754 - https://rest.ensembl.org/sequence/id/ENSMUST00000021399?type=cdna
9755 - https://rest.ensembl.org/sequence/id/ENSMUST00000060645?type=cdna
9756 - https://rest.ensembl.org/sequence/id/ENSMUST00000037115?type=cdna
9757 - https://rest.ensembl.org/sequence/id/ENSMUST00000003635?type=cdna
9758 - https://rest.ensembl.org/sequence/id/ENSMUST00000029786?type=cdna
9759 - https://rest.ensembl.org/sequence/id/ENSMUST00000115205?type=cdna
9760 - https://rest.ensembl.org/sequence/id/ENSMUST00000023769?type=cdna
9761 - https://rest.ensembl.org/sequence/id/ENSMUST00000063814?type=cdna
9762 - https://rest.ensembl.org/sequence/id/ENSMUST

9862 - https://rest.ensembl.org/sequence/id/ENSMUST00000088237?type=cdna
9863 - https://rest.ensembl.org/sequence/id/ENSMUST00000109256?type=cdna
9864 - https://rest.ensembl.org/sequence/id/ENSMUST00000167796?type=cdna
9865 - https://rest.ensembl.org/sequence/id/ENSMUST00000036570?type=cdna
9866 - https://rest.ensembl.org/sequence/id/ENSMUST00000160507?type=cdna
9867 - https://rest.ensembl.org/sequence/id/ENSMUST00000199730?type=cdna
9868 - https://rest.ensembl.org/sequence/id/ENSMUST00000026743?type=cdna
9869 - https://rest.ensembl.org/sequence/id/ENSMUST00000202750?type=cdna
9870 - https://rest.ensembl.org/sequence/id/ENSMUST00000148960?type=cdna
9871 - https://rest.ensembl.org/sequence/id/ENSMUST00000022380?type=cdna
9872 - https://rest.ensembl.org/sequence/id/ENSMUST00000156503?type=cdna
9873 - https://rest.ensembl.org/sequence/id/ENSMUST00000090872?type=cdna
9874 - https://rest.ensembl.org/sequence/id/ENSMUST00000106393?type=cdna
9875 - https://rest.ensembl.org/sequence/id/ENSMUST

In [12]:
save_one_fasta_from_ensembl_api_return(os.path.abspath(r'./samples'), 's1_cdna.fasta', res)

FASTA file: D:\workspace\rnamotif\samples\s1_cdna.fasta


True

In [14]:
res2 = get_ensembl_seq_by_gene(lst_gene2,seq_type='cds',is_ssl_verification=False)

0 - https://rest.ensembl.org/sequence/id/ENSMUST00000026704?type=cds
1 - https://rest.ensembl.org/sequence/id/ENSMUST00000098990?type=cds
2 - https://rest.ensembl.org/sequence/id/ENSMUST00000198271?type=cds
3 - https://rest.ensembl.org/sequence/id/ENSMUST00000088307?type=cds
4 - https://rest.ensembl.org/sequence/id/ENSMUST00000025847?type=cds
5 - https://rest.ensembl.org/sequence/id/ENSMUST00000107348?type=cds
6 - https://rest.ensembl.org/sequence/id/ENSMUST00000043422?type=cds
7 - https://rest.ensembl.org/sequence/id/ENSMUST00000047199?type=cds
8 - https://rest.ensembl.org/sequence/id/ENSMUST00000160510?type=cds
9 - https://rest.ensembl.org/sequence/id/ENSMUST00000111336?type=cds
10 - https://rest.ensembl.org/sequence/id/ENSMUST00000020341?type=cds
11 - https://rest.ensembl.org/sequence/id/ENSMUST00000144843?type=cds
12 - https://rest.ensembl.org/sequence/id/ENSMUST00000025705?type=cds
13 - https://rest.ensembl.org/sequence/id/ENSMUST00000102889?type=cds
14 - https://rest.ensembl.org/

117 - https://rest.ensembl.org/sequence/id/ENSMUST00000127543?type=cds
118 - https://rest.ensembl.org/sequence/id/ENSMUST00000040676?type=cds
119 - https://rest.ensembl.org/sequence/id/ENSMUST00000178385?type=cds
120 - https://rest.ensembl.org/sequence/id/ENSMUST00000110557?type=cds
121 - https://rest.ensembl.org/sequence/id/ENSMUST00000205835?type=cds
122 - https://rest.ensembl.org/sequence/id/ENSMUST00000201050?type=cds
123 - https://rest.ensembl.org/sequence/id/ENSMUST00000034945?type=cds
124 - https://rest.ensembl.org/sequence/id/ENSMUST00000026833?type=cds
125 - https://rest.ensembl.org/sequence/id/ENSMUST00000127215?type=cds
126 - https://rest.ensembl.org/sequence/id/ENSMUST00000076639?type=cds
127 - https://rest.ensembl.org/sequence/id/ENSMUST00000081554?type=cds
128 - https://rest.ensembl.org/sequence/id/ENSMUST00000179990?type=cds
129 - https://rest.ensembl.org/sequence/id/ENSMUST00000146984?type=cds
130 - https://rest.ensembl.org/sequence/id/ENSMUST00000102605?type=cds
131 - 

233 - https://rest.ensembl.org/sequence/id/ENSMUST00000053872?type=cds
234 - https://rest.ensembl.org/sequence/id/ENSMUST00000135469?type=cds
235 - https://rest.ensembl.org/sequence/id/ENSMUST00000051633?type=cds
236 - https://rest.ensembl.org/sequence/id/ENSMUST00000170517?type=cds
237 - https://rest.ensembl.org/sequence/id/ENSMUST00000046383?type=cds
238 - https://rest.ensembl.org/sequence/id/ENSMUST00000024739?type=cds
239 - https://rest.ensembl.org/sequence/id/ENSMUST00000131770?type=cds
240 - https://rest.ensembl.org/sequence/id/ENSMUST00000142140?type=cds
241 - https://rest.ensembl.org/sequence/id/ENSMUST00000163300?type=cds
242 - https://rest.ensembl.org/sequence/id/ENSMUST00000111438?type=cds
243 - https://rest.ensembl.org/sequence/id/ENSMUST00000132804?type=cds
244 - https://rest.ensembl.org/sequence/id/ENSMUST00000128010?type=cds
245 - https://rest.ensembl.org/sequence/id/ENSMUST00000055485?type=cds
246 - https://rest.ensembl.org/sequence/id/ENSMUST00000107720?type=cds
247 - 

349 - https://rest.ensembl.org/sequence/id/ENSMUST00000191493?type=cds
350 - https://rest.ensembl.org/sequence/id/ENSMUST00000033540?type=cds
351 - https://rest.ensembl.org/sequence/id/ENSMUST00000207152?type=cds
352 - https://rest.ensembl.org/sequence/id/ENSMUST00000028403?type=cds
353 - https://rest.ensembl.org/sequence/id/ENSMUST00000045617?type=cds
354 - https://rest.ensembl.org/sequence/id/ENSMUST00000109401?type=cds
355 - https://rest.ensembl.org/sequence/id/ENSMUST00000176553?type=cds
356 - https://rest.ensembl.org/sequence/id/ENSMUST00000138999?type=cds
357 - https://rest.ensembl.org/sequence/id/ENSMUST00000004686?type=cds
358 - https://rest.ensembl.org/sequence/id/ENSMUST00000044011?type=cds
359 - https://rest.ensembl.org/sequence/id/ENSMUST00000163623?type=cds
360 - https://rest.ensembl.org/sequence/id/ENSMUST00000045866?type=cds
361 - https://rest.ensembl.org/sequence/id/ENSMUST00000089230?type=cds
362 - https://rest.ensembl.org/sequence/id/ENSMUST00000057598?type=cds
363 - 

465 - https://rest.ensembl.org/sequence/id/ENSMUST00000039763?type=cds
466 - https://rest.ensembl.org/sequence/id/ENSMUST00000002412?type=cds
467 - https://rest.ensembl.org/sequence/id/ENSMUST00000134082?type=cds
468 - https://rest.ensembl.org/sequence/id/ENSMUST00000117397?type=cds
469 - https://rest.ensembl.org/sequence/id/ENSMUST00000122293?type=cds
470 - https://rest.ensembl.org/sequence/id/ENSMUST00000109737?type=cds
471 - https://rest.ensembl.org/sequence/id/ENSMUST00000153957?type=cds
472 - https://rest.ensembl.org/sequence/id/ENSMUST00000094451?type=cds
473 - https://rest.ensembl.org/sequence/id/ENSMUST00000207879?type=cds
474 - https://rest.ensembl.org/sequence/id/ENSMUST00000081940?type=cds
475 - https://rest.ensembl.org/sequence/id/ENSMUST00000034355?type=cds
476 - https://rest.ensembl.org/sequence/id/ENSMUST00000022614?type=cds
477 - https://rest.ensembl.org/sequence/id/ENSMUST00000164655?type=cds
478 - https://rest.ensembl.org/sequence/id/ENSMUST00000076431?type=cds
479 - 

...

4358 - https://rest.ensembl.org/sequence/id/ENSMUST00000122302?type=cds
4359 - https://rest.ensembl.org/sequence/id/ENSMUST00000027534?type=cds
4360 - https://rest.ensembl.org/sequence/id/ENSMUST00000107771?type=cds
4361 - https://rest.ensembl.org/sequence/id/ENSMUST00000147306?type=cds
4362 - https://rest.ensembl.org/sequence/id/ENSMUST00000023452?type=cds
4363 - https://rest.ensembl.org/sequence/id/ENSMUST00000118286?type=cds
4364 - https://rest.ensembl.org/sequence/id/ENSMUST00000040440?type=cds
4365 - https://rest.ensembl.org/sequence/id/ENSMUST00000170054?type=cds
4366 - https://rest.ensembl.org/sequence/id/ENSMUST00000108141?type=cds
4367 - https://rest.ensembl.org/sequence/id/ENSMUST00000042251?type=cds
4368 - https://rest.ensembl.org/sequence/id/ENSMUST00000109769?type=cds
4369 - https://rest.ensembl.org/sequence/id/ENSMUST00000045621?type=cds
4370 - https://rest.ensembl.org/sequence/id/ENSMUST00000103036?type=cds
4371 - https://rest.ensembl.org/sequence/id/ENSMUST00000110706?t

4472 - https://rest.ensembl.org/sequence/id/ENSMUST00000110057?type=cds
4473 - https://rest.ensembl.org/sequence/id/ENSMUST00000071973?type=cds
4474 - https://rest.ensembl.org/sequence/id/ENSMUST00000029336?type=cds
4475 - https://rest.ensembl.org/sequence/id/ENSMUST00000081640?type=cds
4476 - https://rest.ensembl.org/sequence/id/ENSMUST00000043896?type=cds
4477 - https://rest.ensembl.org/sequence/id/ENSMUST00000171426?type=cds
4478 - https://rest.ensembl.org/sequence/id/ENSMUST00000153944?type=cds
4479 - https://rest.ensembl.org/sequence/id/ENSMUST00000161741?type=cds
4480 - https://rest.ensembl.org/sequence/id/ENSMUST00000202911?type=cds
4481 - https://rest.ensembl.org/sequence/id/ENSMUST00000034570?type=cds
4482 - https://rest.ensembl.org/sequence/id/ENSMUST00000051241?type=cds
4483 - https://rest.ensembl.org/sequence/id/ENSMUST00000145317?type=cds
4484 - https://rest.ensembl.org/sequence/id/ENSMUST00000200914?type=cds
4485 - https://rest.ensembl.org/sequence/id/ENSMUST00000166677?t

4586 - https://rest.ensembl.org/sequence/id/ENSMUST00000134789?type=cds
4587 - https://rest.ensembl.org/sequence/id/ENSMUST00000138231?type=cds
4588 - https://rest.ensembl.org/sequence/id/ENSMUST00000023813?type=cds
4589 - https://rest.ensembl.org/sequence/id/ENSMUST00000194381?type=cds
4590 - https://rest.ensembl.org/sequence/id/ENSMUST00000019302?type=cds
4591 - https://rest.ensembl.org/sequence/id/ENSMUST00000059093?type=cds
4592 - https://rest.ensembl.org/sequence/id/ENSMUST00000060148?type=cds
4593 - https://rest.ensembl.org/sequence/id/ENSMUST00000089759?type=cds
4594 - https://rest.ensembl.org/sequence/id/ENSMUST00000115777?type=cds
4595 - https://rest.ensembl.org/sequence/id/ENSMUST00000033809?type=cds
4596 - https://rest.ensembl.org/sequence/id/ENSMUST00000106349?type=cds
4597 - https://rest.ensembl.org/sequence/id/ENSMUST00000127290?type=cds
4598 - https://rest.ensembl.org/sequence/id/ENSMUST00000207747?type=cds
4599 - https://rest.ensembl.org/sequence/id/ENSMUST00000103204?t

4701 - https://rest.ensembl.org/sequence/id/ENSMUST00000114881?type=cds
4702 - https://rest.ensembl.org/sequence/id/ENSMUST00000039926?type=cds
4703 - https://rest.ensembl.org/sequence/id/ENSMUST00000032761?type=cds
4704 - https://rest.ensembl.org/sequence/id/ENSMUST00000023165?type=cds
4705 - https://rest.ensembl.org/sequence/id/ENSMUST00000174408?type=cds
4706 - https://rest.ensembl.org/sequence/id/ENSMUST00000053491?type=cds
4707 - https://rest.ensembl.org/sequence/id/ENSMUST00000132618?type=cds
4708 - https://rest.ensembl.org/sequence/id/ENSMUST00000024486?type=cds
4709 - https://rest.ensembl.org/sequence/id/ENSMUST00000142468?type=cds
4710 - https://rest.ensembl.org/sequence/id/ENSMUST00000100399?type=cds
4711 - https://rest.ensembl.org/sequence/id/ENSMUST00000021338?type=cds
4712 - https://rest.ensembl.org/sequence/id/ENSMUST00000112250?type=cds
4713 - https://rest.ensembl.org/sequence/id/ENSMUST00000180086?type=cds
4714 - https://rest.ensembl.org/sequence/id/ENSMUST00000079590?t

4815 - https://rest.ensembl.org/sequence/id/ENSMUST00000127694?type=cds
4816 - https://rest.ensembl.org/sequence/id/ENSMUST00000074241?type=cds
4817 - https://rest.ensembl.org/sequence/id/ENSMUST00000043520?type=cds
4818 - https://rest.ensembl.org/sequence/id/ENSMUST00000034512?type=cds
4819 - https://rest.ensembl.org/sequence/id/ENSMUST00000002733?type=cds
4820 - https://rest.ensembl.org/sequence/id/ENSMUST00000002397?type=cds
4821 - https://rest.ensembl.org/sequence/id/ENSMUST00000119633?type=cds
4822 - https://rest.ensembl.org/sequence/id/ENSMUST00000101354?type=cds
4823 - https://rest.ensembl.org/sequence/id/ENSMUST00000030971?type=cds
4824 - https://rest.ensembl.org/sequence/id/ENSMUST00000160439?type=cds
4825 - https://rest.ensembl.org/sequence/id/ENSMUST00000153820?type=cds
4826 - https://rest.ensembl.org/sequence/id/ENSMUST00000082136?type=cds
4827 - https://rest.ensembl.org/sequence/id/ENSMUST00000082409?type=cds
4828 - https://rest.ensembl.org/sequence/id/ENSMUST00000155487?t

In [15]:
save_one_fasta_from_ensembl_api_return(os.path.abspath(r'./samples'), 's1_cds_2.fasta', res2)

FASTA file: D:\workspace\rnamotif\samples\s1_cds_2.fasta


True